# Final Project: PanClassif

Import libraries

In [1]:
# import panclassif as pc
import pandas as pd
import numpy as np
from panclassif import dataProcess, featSelect, upsampled, binary_merge, multi_merge, classification
from sklearn.ensemble import RandomForestClassifier

### Feature Selection

In [2]:
homepath_rare = './rare_data'
cancerpath_rare = './rare_data/cancer'
normalpath_rare = './rare_data/normal'

In [3]:
names_rare = featSelect(homepath_rare,cancerpath_rare,normalpath_rare, k=5)

Cancer files: ['SARC.csv.gz', 'PCPG.csv.gz', 'THYM.csv.gz', 'KICH.csv.gz', 'CHOL.csv.gz']
Normal files: ['PCPG.norm.csv.gz', 'THYM.norm.csv.gz', 'CHOL.norm.csv.gz', 'KICH.norm.csv.gz', 'SARC.norm.csv.gz']
Feature selection process is running...
Processing file index: 0
Doing feature selection for CHOL
Processing file index: 1
Doing feature selection for KICH
Processing file index: 2
Doing feature selection for PCPG
Processing file index: 3
Doing feature selection for SARC
Processing file index: 4
Doing feature selection for THYM
Feature selection process has ended



In [4]:
names_rare

['CHOL', 'KICH', 'PCPG', 'SARC', 'THYM']

In [5]:
# homepath_common = './common_data/'
# cancerpath_common = './common_data/cancer'
# normalpath_common = './common_data/normal'

In [6]:
# names_common = featSelect(homepath_common,cancerpath_common,normalpath_common, k=5)

In [7]:
# names_common

### Data Process

In [8]:
smoothed_cancer_rare = './rare_data/cancer_smoothed'
smoothed_normal_rare = './rare_data/normal_smoothed'

In [9]:
dataProcess(homepath=homepath_rare,
            names=names_rare,
            cancerpath=cancerpath_rare,
            smoothed_cancer=smoothed_cancer_rare,
            smoothed_normal=smoothed_normal_rare,
            scale_mode=0,
            cancer='CHOL')

Data process is running
Unique genes: 25
CHOL
(45, 25)
(9, 25)
KICH
(91, 25)
(25, 25)
PCPG
(187, 25)
(3, 25)
SARC
(265, 25)
(2, 25)
THYM
(122, 25)
(2, 25)
Data process has ended



In [10]:
# smoothed_cancer_common = './common_data/cancer_smoothed'
# smoothed_normal_common = './common_data/normal_smoothed'

In [11]:
# dataProcess(homepath=homepath_common,
#             names=names_common,
#             cancerpath=cancerpath_common,
#             smoothed_cancer=smoothed_cancer_common,
#             smoothed_normal=smoothed_normal_common,
#             scale_mode=0,
#             cancer='BRCA')

Upsampling

In [12]:
upsampled(names_rare, homepath_rare)

Upsampling on train data is running
Upsampling on train data has ended



In [13]:
# upsampled(names_common, homepath_common)

Binary merge

In [14]:
binary_merge(names_rare, homepath_rare)

Merging for binary has started
Merging for binary has ended



In [15]:
# binary_merge(names_common, homepath_common)

Multi-merge

In [16]:
multi_merge(names_rare, homepath_rare)

Merging for multiclass has started
Merging for multiclass has ended



In [17]:
# multi_merge(names_common, homepath_common)

Binary Classification

In [18]:
# classification(homepath_rare, RandomForestClassifier(n_estimators=100), mode="binary", save_model="RF")

In [19]:
# classification(homepath_common, RandomForestClassifier(n_estimators=100), mode="binary", save_model="RF")

Multi-classification

In [20]:
# classification(homepath_rare, RandomForestClassifier(n_estimators=100), mode="multi", save_model="RF")

In [21]:
# classification(homepath_common, RandomForestClassifier(n_estimators=100), mode="multi", save_model="RF")

Gene enrichment check

In [22]:
# gsea(homepath_rare)

## Try ncv with rare dataset

In [23]:
from panclassif.ncvclass import NestedCV

In [24]:
#train data load
Cancer = pd.read_csv(homepath_rare+"/train_data/bin_Cancer.txt.bz2",header=None, delimiter = "\t")
Normal = pd.read_csv(homepath_rare+"/train_data/bin_Normal.txt.bz2",header=None, delimiter = "\t")
Cancer['Target'] = 1
Normal['Target'] = 0
#Normal = Normal.drop(Normal.index[0])
frame = [Cancer,Normal]
Data = pd.concat(frame,axis=0)
X_train = Data.iloc[:,:len(Data.columns)-1]
y_train = Data.iloc[:,len(Data.columns)-1]

In [28]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import optuna
import optuna.integration.sklearn

# Assuming X_train and y_train are pandas DataFrames/Series
# Reset index if there are issues with non-sequential indices
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

# Replace NaN values with the mean of the column
imputer = SimpleImputer(strategy='mean')
X_train_clean = imputer.fit_transform(X_train)

# Define classifiers and their parameter grids
classifiers = {
    'RandomForest': RandomForestClassifier()
}

# Define Optuna distributions
param_distributions = {
    'n_estimators': optuna.distributions.FloatDistribution(100, 2000)
}

# Create the random grid
param_grids = {'RandomForest': param_distributions}

# Define seeds for reproducibility
seeds = np.random.randint(0, 100, size=30)  # 10 rounds * 3 seeds per round

# Instantiate NestedCV and perform nested cross-validation with Optuna
try:
    nested_cv = NestedCV(classifiers=classifiers, seeds=seeds, param_grids=param_grids)
    all_outer_scores, all_outer_params = nested_cv.fit(X_train_clean, y_train)
    print(all_outer_scores)
    print(all_outer_params)
except ValueError as e:
    print(f"Error during nested cross-validation: {str(e)}")
except Exception as e:
    print(f"Unexpected error: {str(e)}")



Using classifier: RandomForest



  0%|          | 0/10 [00:00<?, ?it/s]


Starting round 1...


Trial 0 failed with parameters: {'n_estimators': 762.8033556884955} because of the following error: The value nan is not acceptable.
Trial 1 failed with parameters: {'n_estimators': 1925.6090254311923} because of the following error: The value nan is not acceptable.
Trial 2 failed with parameters: {'n_estimators': 1207.9447342544934} because of the following error: The value nan is not acceptable.
Trial 0 failed with value nan.
Trial 5 failed with parameters: {'n_estimators': 1645.3292866384136} because of the following error: The value nan is not acceptable.
Trial 1 failed with value nan.
Trial 6 failed with parameters: {'n_estimators': 1850.7654127863075} because of the following error: The value nan is not acceptable.
Trial 4 failed with parameters: {'n_estimators': 426.911862188384} because of the following error: The value nan is not acceptable.
Trial 7 failed with parameters: {'n_estimators': 1440.676540362989} because of the following error: The value nan is not acceptable.
Tria

Unexpected error: 'OptunaSearchCV' object has no attribute 'params'


In [30]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
import optuna.integration.sklearn

# Assuming X_train and y_train are pandas DataFrames/Series
# Reset index if there are issues with non-sequential indices
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

# Replace NaN values with the mean of the column
imputer = SimpleImputer(strategy='mean')
X_train_clean = imputer.fit_transform(X_train)

# Define classifier and its parameter grid
classifier = RandomForestClassifier()

# Define Optuna distributions
param_distributions = {
    'n_estimators': optuna.distributions.FloatDistribution(100, 2000)
}

# Instantiate OptunaSearchCV
optuna_search = optuna.integration.sklearn.OptunaSearchCV(
    classifier,
    param_distributions,
    cv=5,
    n_trials=10,  # Number of trials for optimization
    n_jobs=-1  # Use all available cores
)

# Fit OptunaSearchCV
try:
    optuna_search.fit(X_train_clean, y_train)
    print("Best parameters found:", optuna_search.best_params_)
    print("Best score found:", optuna_search.best_score_)
except Exception as e:
    print(f"Unexpected error during optimization: {str(e)}")


Trial 0 failed with parameters: {'n_estimators': 272.6480359131639} because of the following error: The value nan is not acceptable.
Trial 0 failed with value nan.
Trial 1 failed with parameters: {'n_estimators': 1182.6325814994746} because of the following error: The value nan is not acceptable.
Trial 2 failed with parameters: {'n_estimators': 125.97766025340688} because of the following error: The value nan is not acceptable.
Trial 5 failed with parameters: {'n_estimators': 415.16407194713645} because of the following error: The value nan is not acceptable.
Trial 5 failed with value nan.
Trial 1 failed with value nan.
Trial 4 failed with parameters: {'n_estimators': 659.0293997193166} because of the following error: The value nan is not acceptable.
Trial 3 failed with parameters: {'n_estimators': 1030.8955245787006} because of the following error: The value nan is not acceptable.
Trial 2 failed with value nan.
Trial 7 failed with parameters: {'n_estimators': 903.933747143121} because

Unexpected error during optimization: No trials are completed yet.


In [31]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import optuna
import optuna.visualization as vis

# Load dataset
california = fetch_california_housing()
X = california.data
y = california.target

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define objective function
def objective(trial):
    # Suggest values for hyperparameters
    n_estimators = trial.suggest_int("n_estimators", 10, 200, log=True)
    max_depth = trial.suggest_int("max_depth", 2, 32)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)

    # Create and fit random forest model
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42,
    )
    model.fit(X_train, y_train)

    # Make predictions and calculate RMSE
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Return MAE
    return mae

# Create study object
study = optuna.create_study(direction="minimize")

# Run optimization process
study.optimize(objective, n_trials=20, show_progress_bar=True)

# Print best trial and best hyperparameters
print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)

# Visualize optimization history
vis.plot_optimization_history(study)

# Visualize parameter importance
vis.plot_param_importances(study)

# Visualize slice plot
vis.plot_slice(study, params=["n_estimators", "max_depth"])

# Visualize contour plot
vis.plot_contour(study, params=["min_samples_split", "min_samples_leaf"])

# Visualize parallel coordinate plot
vis.plot_parallel_coordinate(study)



  0%|          | 0/20 [00:00<?, ?it/s]

Best trial: FrozenTrial(number=10, state=1, values=[0.3300254618374199], datetime_start=datetime.datetime(2024, 7, 3, 0, 11, 3, 683429), datetime_complete=datetime.datetime(2024, 7, 3, 0, 11, 22, 119159), params={'n_estimators': 187, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 4}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=200, log=True, low=10, step=1), 'max_depth': IntDistribution(high=32, log=False, low=2, step=1), 'min_samples_split': IntDistribution(high=10, log=False, low=2, step=1), 'min_samples_leaf': IntDistribution(high=10, log=False, low=1, step=1)}, trial_id=10, value=None)
Best hyperparameters: {'n_estimators': 187, 'max_depth': 20, 'min_samples_split': 6, 'min_samples_leaf': 4}
